"Exploratorio"

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

file_path = 'health_data.csv'
df = pd.read_csv(file_path)

display(df.head())

,Edad,Género,Estado civil,Altura,Peso,Índice de masa corporal,¿Fuma actualmente?,¿Fumó en el pasado?,¿Consume alcohol frecuentemente?,Nivel de actividad física,...,¿Sufre de problemas de visión?,¿Tiene problemas de audición?,¿Ha sufrido de fracturas óseas en el pasado?,Nivel de satisfacción con la vida,Enfermedad cardiovascular,Diabetes,Asma,Cáncer,Obesidad,Depresión/Ansiedad
0,76.596326,Otro,Soltero,153.681426,76.920289,29.612895,No,Sí,No,Moderado,...,No,No,No,Medio,0.674302,-0.171059,1.142946,0.293202,-0.130552,0.228336
1,79.795297,Otro,Casado,155.882307,66.743641,9.902543,No,No,No,Moderado,...,Sí,No,Sí,Bajo,-0.014915,0.101641,-0.059422,1.047785,0.216788,1.211533
2,90.603394,Otro,Casado,176.481841,124.818134,27.248719,Sí,No,Sí,Moderado,...,No,No,No,Medio,0.981927,0.054446,0.918964,0.138127,-0.030003,-0.205018
3,22.154276,Femenino,Viudo,158.681358,114.807668,27.634473,No,No,No,Moderado,...,No,Sí,No,Bajo,1.147131,0.256350,-0.159599,-0.260462,1.363624,0.291855
4,46.176676,Masculino,Casado,184.451263,60.217207,24.094841,No,Sí,No,Sedentario,...,No,No,No,Medio,1.067995,0.225792,0.165198,0.015367,0.960565,1.427700


In [3]:
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

Missing values per column:
Series([], dtype: int64)

Number of duplicate rows: 0


# Transformaciones

In [4]:
import pandas as pd

# En este punto supongo que df ya está cargado:
# df = pd.read_csv("health_data.csv")

# Diccionario con las medias y desviaciones estándar que nos dieron
# para cada enfermedad. Lo uso para "deshacer" la estandarización.
stats = {
    "Enfermedad cardiovascular": {"mean": 0.303130, "std": 0.514926},
    "Diabetes": {"mean": 0.205536, "std": 0.450558},
    "Asma": {"mean": 0.150921, "std": 0.399708},
    "Cáncer": {"mean": 0.105589, "std": 0.342888},
    "Obesidad": {"mean": 0.248486, "std": 0.483748},
    "Depresión/Ansiedad": {"mean": 0.401899, "std": 0.545406}
}

# 1) Desestandarizo las columnas de enfermedades para volver a la escala original
for col, info in stats.items():
    # nueva columna *_real con el valor "deshecho" de la estandarización
    df[col + "_real"] = df[col] * info["std"] + info["mean"]

# 2) Paso cada enfermedad a binaria:
#    1 si la probabilidad es mayor o igual a 0.5, 0 si es menor
for col in stats:
    df[col + "_bin"] = (df[col + "_real"] >= 0.5).astype(int)

# Solo para chequear rápido cómo queda el dataframe
df.head()



,Edad,Género,Estado civil,Altura,Peso,Índice de masa corporal,¿Fuma actualmente?,¿Fumó en el pasado?,¿Consume alcohol frecuentemente?,Nivel de actividad física,...,Asma_real,Cáncer_real,Obesidad_real,Depresión/Ansiedad_real,Enfermedad cardiovascular_bin,Diabetes_bin,Asma_bin,Cáncer_bin,Obesidad_bin,Depresión/Ansiedad_bin
0,76.596326,Otro,Soltero,153.681426,76.920289,29.612895,No,Sí,No,Moderado,...,0.607766,0.206125,0.185332,0.526435,1,0,1,0,0,1
1,79.795297,Otro,Casado,155.882307,66.743641,9.902543,No,No,No,Moderado,...,0.127169,0.464862,0.353357,1.062676,0,0,0,0,0,1
2,90.603394,Otro,Casado,176.481841,124.818134,27.248719,Sí,No,Sí,Moderado,...,0.518238,0.152951,0.233972,0.290081,1,0,1,0,0,0
3,22.154276,Femenino,Viudo,158.681358,114.807668,27.634473,No,No,No,Moderado,...,0.087128,0.016280,0.908137,0.561079,1,0,0,0,1,1
4,46.176676,Masculino,Casado,184.451263,60.217207,24.094841,No,Sí,No,Sedentario,...,0.216952,0.110858,0.713157,1.180575,1,0,0,0,1,1


In [5]:
# Me quedo con solo las columnas "reales" de las enfermedades
disease_cols = [
    "Enfermedad cardiovascular_real",
    "Diabetes_real",
    "Asma_real",
    "Cáncer_real",
    "Obesidad_real",
    "Depresión/Ansiedad_real"
]

df_diseases = df[disease_cols].copy()

# Guardo  en un CSV 
df_diseases.to_csv("diseases_only.csv", index=False, encoding="utf-8")


In [6]:
import pyreadr

pyreadr.write_rdata("diseases_only.RData", df_diseases, "diseases")

In [7]:
import numpy as np

# Columnas binarias de las 6 enfermedades (las que ya creaste)
disease_bin_cols = [
    "Enfermedad cardiovascular_bin",
    "Diabetes_bin",
    "Asma_bin",
    "Cáncer_bin",
    "Obesidad_bin",
    "Depresión/Ansiedad_bin"
]


df["disease_combo"] = df[disease_bin_cols].apply(lambda fila: tuple(fila.values), axis=1)


combo_counts = df["disease_combo"].value_counts()

min_count = 30   # si quieres menos clases, sube este número

combos_comunes = combo_counts[combo_counts >= min_count].index

def agrupar_combos(combo):
    if combo in combos_comunes:
        return combo
    else:
        return "OTRAS"

# Versión reducida de la combinación, con las raras agrupadas
df["disease_combo_reduced"] = df["disease_combo"].apply(agrupar_combos)

# Ahora paso estas etiquetas (tuplas + "OTRAS") a números 0,1,2,...
etiquetas_unicas = list(df["disease_combo_reduced"].unique())
label_to_id = {etq: i for i, etq in enumerate(etiquetas_unicas)}

df["disease_class_final"] = df["disease_combo_reduced"].map(label_to_id)

df[["disease_combo", "disease_combo_reduced", "disease_class_final"]].head()


,disease_combo,disease_combo_reduced,disease_class_final
0,"(1, 0, 1, 0, 0, 1)","(1, 0, 1, 0, 0, 1)",0
1,"(0, 0, 0, 0, 0, 1)","(0, 0, 0, 0, 0, 1)",1
2,"(1, 0, 1, 0, 0, 0)","(1, 0, 1, 0, 0, 0)",2
3,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3
4,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3


## Introducción a los modelos basados en redes neuronales

A partir del análisis exploratorio de los datos de salud, el objetivo de esta parte del trabajo fue
construir y comparar distintos modelos predictivos capaces de identificar la presencia de
enfermedades (o combinaciones de ellas) a partir de variables demográficas, de estilo de vida y
antecedentes clínicos.  

Dado que el conjunto de datos es relativamente grande y las relaciones entre variables pueden ser
no lineales, optamos por utilizar **redes neuronales artificiales** como modelo base. A partir de
esta idea probamos varias variantes:

1. **Red neuronal multiclase básica**  
   Primero definimos un problema de clasificación multiclase donde la variable objetivo
   `disease_class_final` agrupa las distintas combinaciones de enfermedades.  
   Entrenamos una red densa con dos capas ocultas (64 y 32 neuronas con activación ReLU y
   dropout), salida softmax y pérdida `categorical_crossentropy`. Este modelo nos sirve como
   punto de partida para evaluar qué tan bien se pueden distinguir las combinaciones de
   enfermedades usando solo las variables de entrada disponibles.

2. **Bagging con redes neuronales**  
   Para intentar reducir la varianza del modelo y ganar estabilidad, construimos un
   **ensemble** mediante bagging: entrenamos varias redes con la misma arquitectura sobre
   distintos bootstraps del conjunto de entrenamiento y promediamos sus predicciones.  
   Probamos tanto un bagging “simple” como una variante con bootstraps balanceados por clase,
   con la idea de darle más presencia a las clases minoritarias durante el entrenamiento.

3. **Red neuronal multiclase con pesos de clase**  
   Dado el fuerte desbalance entre combinaciones de enfermedades, incorporamos también un
   modelo multiclase con **`class_weight`**, penalizando más los errores en las clases raras.
   Este modelo permite comprobar hasta qué punto se puede mejorar el F1-macro ajustando el
   coste de las clases en la función de pérdida, sin cambiar la arquitectura de la red.

4. **Red neuronal multi-etiqueta (una salida por enfermedad)**  
   Finalmente replanteamos el problema como **clasificación multi-etiqueta**, prediciendo en
   paralelo seis variables binarias (`*_bin`) que indican la presencia de enfermedad
   cardiovascular, diabetes, asma, cáncer, obesidad y depresión/ansiedad.  
   Para ello usamos la misma estructura de red, pero con una capa de salida de seis neuronas
   sigmoides y pérdida `binary_crossentropy`, lo que nos permite analizar el rendimiento
   individual por enfermedad y no solo por combinación.

En las secciones siguientes describimos en detalle cada uno de estos modelos, los parámetros
utilizados (arquitectura, épocas, tamaño de batch, etc.) y las métricas obtenidas (accuracy,
F1-macro y F1 por enfermedad). Esto nos permite entender no solo qué modelo funciona mejor, sino
también hasta qué punto el **desbalance de clases y la información disponible en las variables**
limitan el rendimiento de las redes neuronales en este problema.


# Preparar X e y y seguir con la red neuronal

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score

# 1) columnas de enfermedades
disease_bin_cols = [
    "Enfermedad cardiovascular_bin",
    "Diabetes_bin",
    "Asma_bin",
    "Cáncer_bin",
    "Obesidad_bin",
    "Depresión/Ansiedad_bin"
]

disease_cont_cols = [
    "Enfermedad cardiovascular",
    "Diabetes",
    "Asma",
    "Cáncer",
    "Obesidad",
    "Depresión/Ansiedad"
]

disease_real_cols = [c + "_real" for c in disease_cont_cols]

target_col = "disease_class_final"

cols_a_sacar = (
    disease_bin_cols
    + disease_cont_cols
    + disease_real_cols
    + ["disease_combo", "disease_combo_reduced",
       "disease_class", "disease_class_reduced", "disease_class_final"]
)

cols_a_sacar = [c for c in cols_a_sacar if c in df.columns]

X = df.drop(columns=cols_a_sacar)
y = df[target_col]

print("Shape X:", X.shape)
print("Clases distintas en y:", y.nunique())


Shape X: (20000, 50)
Clases distintas en y: 34


In [20]:

cat_cols = X.select_dtypes(include=["object"]).columns
print("Columnas categóricas:", list(cat_cols))

X = pd.get_dummies(X, columns=cat_cols, drop_first=False, dtype=int)
print("Shape X después de dummies:", X.shape)


Columnas categóricas: ['Género', 'Estado civil', '¿Fuma actualmente?', '¿Fumó en el pasado?', '¿Consume alcohol frecuentemente?', 'Nivel de actividad física', '¿Tiene una dieta equilibrada?', '¿Consume frutas y verduras diariamente?', 'Frecuencia de consumo de comida rápida', '¿Duerme al menos 7 horas por noche?', '¿Experimenta estrés con frecuencia?', '¿Tiene antecedentes de hipertensión en la familia?', '¿Tiene antecedentes de diabetes en la familia?', '¿Tiene antecedentes de cáncer en la familia?', '¿Tiene antecedentes de enfermedades cardiovasculares en la familia?', '¿Tiene antecedentes de problemas de tiroides en la familia?', 'Frecuencia de ejercicio físico semanal', '¿Toma medicamentos regularmente?', 'Nivel de colesterol', 'Nivel de triglicéridos', 'Nivel de glucosa en sangre', 'Presión arterial', 'Consumo de sal en la dieta', '¿Tiene antecedentes de obesidad en la familia?', '¿Tiene antecedentes de asma?', '¿Padece de alguna alergia?', '¿Ha tenido infecciones respiratorias fr

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)



### Configuración del modelo 1 de red neuronal

Para el modelo de redes neuronales usamos la siguiente arquitectura y parámetros:

- **Arquitectura `Sequential`**  
  Elegimos un modelo `Sequential` porque nuestro problema es un flujo sencillo de entrada → salida, sin conexiones raras ni múltiples entradas. Esto hace el código más simple y suficiente para una tarea de clasificación.

- **Capas densas de 64 y 32 neuronas con `relu`**  
  Usamos dos capas ocultas:
  - Primera capa: `Dense(64, activation = "relu")`  
  - Segunda capa: `Dense(32, activation = "relu")`  
  El tamaño 64–32 es un compromiso entre capacidad y sobreajuste: da suficientes neuronas para capturar relaciones no lineales entre las variables, pero sin ser tan grande como para memorizar el dataset. La activación `relu` se usa porque suele converger rápido y funciona bien en problemas de clasificación tabular.

- **Capas `Dropout(0.3)`**  
  Después de cada capa densa añadimos `Dropout(0.3)`, es decir, apagamos aleatoriamente el 30% de las neuronas en cada paso de entrenamiento. Esto ayuda a **reducir el sobreajuste**, obligando al modelo a no depender demasiado de unas pocas neuronas y a generalizar mejor al conjunto de test.

- **Capa de salida con `num_classes` neuronas y `softmax`**  
  La última capa tiene tantas neuronas como clases (`num_classes`) y usa activación `softmax`.  
  Esto nos da, para cada observación, una distribución de probabilidades sobre todas las clases posibles y es el estándar en problemas de **clasificación multiclase**.

- **Codificación one-hot del objetivo (`to_categorical`)**  
  Convertimos `y_train` e `y_test` a formato one-hot con `to_categorical`. Así cada clase se representa como un vector binario, compatible con la salida `softmax` y la pérdida `categorical_crossentropy`.

- **Función de pérdida: `categorical_crossentropy`**  
  Usamos `categorical_crossentropy` porque es la función de pérdida habitual cuando:
  1) la variable objetivo tiene más de dos clases, y  
  2) la salida del modelo es una probabilidad por clase (softmax).

- **Optimizador `adam`**  
  Elegimos `adam` porque es un optimizador robusto y automático: ajusta la tasa de aprendizaje internamente y suele dar buen rendimiento sin mucho ajuste fino, lo cual es práctico para un proyecto aplicado como este.

- **Número de épocas: `epochs = 30`**  
  Entrenamos durante 30 épocas como punto medio: suficiente para que el modelo aprenda los patrones principales sin entrenar “eternamente”. Además, al usar `validation_split = 0.2` podemos ver si la pérdida de validación deja de mejorar y, si hiciera falta, podríamos reducir o aumentar este valor.

- **Tamaño de batch: `batch_size = 128`**  
  Un batch de 128 observaciones equilibra estabilidad en el gradiente y tiempo de cómputo:  
  batches muy pequeños hacen el entrenamiento ruidoso; batches enormes pueden ser más lentos y consumir más memoria. 128 es un valor estándar que funciona bien en la práctica.

- **`validation_split = 0.2`**  
  Reservamos el 20% del conjunto de entrenamiento como validación interna. Esto nos permite monitorizar si el modelo empieza a sobreajustar (cuando la pérdida de validación empeora mientras la de entrenamiento mejora) sin tocar todavía el conjunto de test.

En resumen, elegimos una arquitectura relativamente pequeña (64–32 neuronas) con `relu` y `dropout` como compromiso entre capacidad de modelar relaciones no lineales y control del sobreajuste, utilizando los hiperparámetros estándar (`adam`, `categorical_crossentropy`, `batch_size = 128`, `epochs = 30`) para un problema de clasificación multiclase.


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

num_features = X_train_scaled.shape[1]
num_classes  = y_train.nunique()

# Paso y a one-hot para usar softmax
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat  = to_categorical(y_test,  num_classes=num_classes)

model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(num_features,)))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation="softmax"))

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 34)             │         1,122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,178 (39.76 KB)

 Trainable params: 10,178 (39.76 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    X_train_scaled,
    y_train_cat,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
100/100 - 1s - 15ms/step - accuracy: 0.1337 - loss: 3.2658 - val_accuracy: 0.1672 - val_loss: 2.9260
Epoch 2/30
100/100 - 0s - 1ms/step - accuracy: 0.1620 - loss: 2.9604 - val_accuracy: 0.1722 - val_loss: 2.8599
Epoch 3/30
100/100 - 0s - 1ms/step - accuracy: 0.1756 - loss: 2.9041 - val_accuracy: 0.1756 - val_loss: 2.8447
Epoch 4/30
100/100 - 0s - 1ms/step - accuracy: 0.1804 - loss: 2.8655 - val_accuracy: 0.1769 - val_loss: 2.8401
Epoch 5/30
100/100 - 0s - 1ms/step - accuracy: 0.1830 - loss: 2.8497 - val_accuracy: 0.1791 - val_loss: 2.8367
Epoch 6/30
100/100 - 0s - 1ms/step - accuracy: 0.1847 - loss: 2.8357 - val_accuracy: 0.1713 - val_loss: 2.8370
Epoch 7/30
100/100 - 0s - 1ms/step - accuracy: 0.1814 - loss: 2.8212 - val_accuracy: 0.1741 - val_loss: 2.8349
Epoch 8/30
100/100 - 0s - 1ms/step - accuracy: 0.1827 - loss: 2.8163 - val_accuracy: 0.1787 - val_loss: 2.8360
Epoch 9/30
100/100 - 0s - 1ms/step - accuracy: 0.1900 - loss: 2.8036 - val_accuracy: 0.1694 - val_loss: 2.8356


### Evolución del entrenamiento

Durante las 30 épocas de entrenamiento se observa:

- La **accuracy de entrenamiento** parte alrededor de 0.13 y sube muy poco, hasta ~0.19.
- La **accuracy de validación** se mantiene también baja y bastante estable, en el rango 0.16–0.18.
- La **loss** y la **val_loss** bajan ligeramente al principio y luego se estabilizan, sin señales claras de sobreajuste (no hay una separación grande entre loss de train y de validación).

Esto sugiere que el modelo **no está aprendiendo patrones muy fuertes** a partir de las variables disponibles: el rendimiento se estanca pronto y nunca llega a valores altos de exactitud. Es decir, el problema es realmente difícil si no le damos al modelo las enfermedades originales como entrada.


In [24]:
y_pred_proba = model.predict(X_test_scaled)
y_pred_class = y_pred_proba.argmax(axis=1)

print("F1 macro (Red Neuronal):",
      f1_score(y_test, y_pred_class, average="macro"))

print(classification_report(y_test, y_pred_class))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
F1 macro (Red Neuronal): 0.01417706885474748
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.19      0.84      0.31       747
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00       133
           4       0.00      0.00      0.00        63
           5       0.16      0.17      0.16       635
           6       1.00      0.00      0.01       329
           7       0.00      0.00      0.00        46
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00        10
          10       0.00      0.00      0.00       190
          11       0.00      0.00      0.00       261
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00        75
          14       0.00      0.00      0.00        32
          15       0.00      0.00      0.00    

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

### Métricas finales en el conjunto de test

El *classification report* sobre el conjunto de test muestra:

- **Accuracy global ≈ 0.18**, lo que significa que el modelo acierta aproximadamente un 18% de los casos.
- **F1 macro ≈ 0.01**, muy bajo: indica que, en promedio, las clases se predicen mal.  
  La red tiende a concentrarse en unas pocas clases frecuentes y prácticamente ignora las raras.
- En las métricas por clase se ve que:
  - La clase más frecuente (por ejemplo la clase 1) tiene un *recall* alto (≈0.84), pero una precisión baja (≈0.19): el modelo casi siempre “se tira” a esa clase y acierta solo cuando realmente corresponde.
  - Para muchas clases minoritarias, la precisión y el recall son 0: el modelo casi nunca las predice.

En conclusión, después de eliminar la fuga de información (las columnas de enfermedades) la red neuronal **pierde la performance perfecta que tenía antes** y pasa a tener un rendimiento modesto, especialmente en las clases poco frecuentes. Esto es coherente con un problema desbalanceado y con variables predictoras que no determinan de forma directa la combinación de enfermedades. Sirve también para mostrar que, sin hacer trampa con las variables de entrada, el modelo tiene limitaciones claras y habría que explorar más técnicas (re-balanceo, otras arquitecturas, más features) si se quisiera mejorar su F1 macro.

# Baggin con Red Neuronal


### Bagging con redes neuronales

Además del modelo de red neuronal individual, probamos un esquema de **bagging** (Bootstrap Aggregating) usando varias redes neuronales con la misma arquitectura.

La idea del bagging es reducir la **varianza** del modelo: en lugar de entrenar una única red sobre todo el conjunto de entrenamiento, entrenamos varias redes sobre diferentes muestras bootstrap y luego promediamos sus predicciones.

#### Arquitectura de cada red del ensemble

Primero definimos una función `crear_modelo_nn()` que construye una red con la misma estructura que el modelo base:

- Capa densa de **64 neuronas** con activación *ReLU*.
- Capa de **Dropout(0.3)** para reducir sobreajuste.
- Segunda capa densa de **32 neuronas** con activación *ReLU*.
- Otra capa de **Dropout(0.3)**.
- Capa de salida con `num_classes` neuronas y activación *softmax* (una probabilidad por cada combinación de enfermedades).

La red se compila con:

- **Optimizador:** `adam`, por su buen comportamiento general sin mucho ajuste fino.
- **Pérdida:** `categorical_crossentropy`, apropiada para clasificación multiclase con salida *softmax*.
- **Métrica:** `accuracy` para monitorizar el entrenamiento.

De esta forma todas las redes del ensemble comparten la misma arquitectura y difieren solo en los datos concretos sobre los que se entrenan.

#### Procedimiento de bagging

Para el bagging usamos los siguientes parámetros:

- **`n_models = 5`**  
  Entrenamos 5 redes distintas. Es un número moderado que permite ver el efecto del ensamble sin disparar el tiempo de cómputo. Si se quisiera un ensemble más fuerte se podría aumentar este valor.

- **`epocas_por_modelo = 20`**  
  Cada red se entrena durante 20 épocas (algo menos que el modelo individual) para mantener razonable el tiempo total de entrenamiento, ya que ahora hay que entrenar varios modelos.

El procedimiento dentro del bucle es:

1. **Muestreo bootstrap:**  
   Para cada modelo generamos una muestra bootstrap del conjunto de entrenamiento (`X_train_scaled`, `y_train`).  
   Esto significa que seleccionamos aleatoriamente, con reemplazo, tantas filas como el tamaño del train original. Cada red ve una versión ligeramente distinta de los datos.

2. **Codificación one-hot del objetivo:**  
   Convertimos `y_boot` a formato one-hot (`y_boot_cat`) con `to_categorical`, igual que en el modelo base, para poder usar *softmax* + `categorical_crossentropy`.

3. **Entrenamiento del modelo:**  
   Creamos un nuevo modelo con `crear_modelo_nn()` y lo entrenamos sobre el bootstrap correspondiente (`X_boot`, `y_boot_cat`) durante `epocas_por_modelo` épocas, con `batch_size = 128`.

4. **Almacenamiento de modelos:**  
   Cada modelo entrenado se guarda en la lista `modelos_bagging` para usarlo después en la fase de predicción.




In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

num_features = X_train_scaled.shape[1]
num_classes  = y_train.nunique()

def crear_modelo_nn():
    """
    Arma y compila una red neuronal con la arquitectura que queremos usar
    en cada modelo del bagging.
    """
    model = Sequential()
    model.add(Dense(64, activation="relu", input_shape=(num_features,)))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [29]:
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat  = to_categorical(y_test,  num_classes=num_classes)

model_base = crear_modelo_nn()
model_base.summary()

history = model_base.fit(
    X_train_scaled,
    y_train_cat,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 64)             │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 34)             │         1,122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,178 (39.76 KB)

 Trainable params: 10,178 (39.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
100/100 - 2s - 16ms/step - accuracy: 0.1268 - loss: 3.2186 - val_accuracy: 0.1706 - val_loss: 2.9251
Epoch 2/30
100/100 - 0s - 1ms/step - accuracy: 0.1697 - loss: 2.9596 - val_accuracy: 0.1775 - val_loss: 2.8684
Epoch 3/30
100/100 - 0s - 1ms/step - accuracy: 0.1705 - loss: 2.8996 - val_accuracy: 0.1719 - val_loss: 2.8531
Epoch 4/30
100/100 - 0s - 1ms/step - accuracy: 0.1819 - loss: 2.8667 - val_accuracy: 0.1797 - val_loss: 2.8455
Epoch 5/30
100/100 - 0s - 1ms/step - accuracy: 0.1852 - loss: 2.8443 - val_accuracy: 0.1816 - val_loss: 2.8402
Epoch 6/30
100/100 - 0s - 1ms/step - accuracy: 0.1882 - loss: 2.8331 - val_accuracy: 0.1819 - val_loss: 2.8405
Epoch 7/30
100/100 - 0s - 1ms/step - accuracy: 0.1858 - loss: 2.8205 - val_accuracy: 0.1838 - val_loss: 2.8370
Epoch 8/30
100/100 - 0s - 1ms/step - accuracy: 0.1854 - loss: 2.8165 - val_accuracy: 0.1834 - val_loss: 2.8381
Epoch 9/30
100/100 - 0s - 1ms/step - accuracy: 0.1886 - loss: 2.8080 - val_accuracy: 0.1853 - val_loss: 2.8344


#### Entrenamiento de las redes dentro del bagging

Cada uno de los modelos del ensemble utiliza la misma arquitectura:

- Capa densa de 64 neuronas con activación *ReLU*.
- Capa de *Dropout(0.3)* para regularizar.
- Capa densa de 32 neuronas con activación *ReLU*.
- Segunda capa de *Dropout(0.3)*.
- Capa de salida con 34 neuronas y activación *softmax*, una por cada clase de `disease_class_final`.

El resumen del modelo indica un total de **10.178 parámetros entrenables**, por lo que cada red del bagging tiene una capacidad moderada: suficiente para capturar relaciones no lineales, pero sin llegar a ser un modelo gigante.

Durante el entrenamiento (30 épocas) se observa que:

- La **accuracy de entrenamiento** empieza alrededor de 0.12 y sube lentamente hasta ~0.19.
- La **accuracy de validación** se mantiene en torno a 0.17–0.18 durante casi todas las épocas.
- La **pérdida de entrenamiento y de validación** disminuyen ligeramente al principio y luego se estabilizan, sin que aparezca una separación grande entre ambas.

Este comportamiento es muy parecido al del modelo de red neuronal individual: el modelo aprende algo de estructura en los datos, pero no consigue una precisión alta y no parece haber sobreajuste fuerte. En otras palabras, incluso entrenando varias redes con esta arquitectura, el problema sigue siendo difícil de resolver únicamente a partir de las variables de entrada disponibles (sin usar directamente las enfermedades originales).


In [30]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

n_models = 5
epocas_por_modelo = 20
modelos_bagging = []

for i in range(n_models):
    print(f"\nEntrenando modelo {i+1}/{n_models} del bagging...")

    # Bootstrap sobre el train
    indices = np.random.choice(len(X_train_scaled), size=len(X_train_scaled), replace=True)
    X_boot = X_train_scaled[indices]
    y_boot = y_train.iloc[indices]

    # One-hot SOLO para este bootstrap
    y_boot_cat = to_categorical(y_boot, num_classes=num_classes)

    # Modelo nuevo para este bootstrap
    m = crear_modelo_nn()
    m.fit(
        X_boot,
        y_boot_cat,
        epochs=epocas_por_modelo,
        batch_size=128,
        verbose=0
    )

    modelos_bagging.append(m)




Entrenando modelo 1/5 del bagging...

Entrenando modelo 2/5 del bagging...

Entrenando modelo 3/5 del bagging...

Entrenando modelo 4/5 del bagging...

Entrenando modelo 5/5 del bagging...


In [31]:
# Predicciones de cada red
preds_proba = []

for i, m in enumerate(modelos_bagging):
    print(f"Prediciendo con modelo {i+1}/{n_models}...")
    p = m.predict(X_test_scaled, verbose=0)
    preds_proba.append(p)

# Promedio de probabilidades entre todos los modelos
preds_proba = np.array(preds_proba)          # shape: (n_models, n_muestras, num_classes)
mean_proba = preds_proba.mean(axis=0)        # shape: (n_muestras, num_classes)

y_pred_bagging_nn = mean_proba.argmax(axis=1)

print("F1 macro (bagging de redes):",
      f1_score(y_test, y_pred_bagging_nn, average="macro"))

print(classification_report(y_test, y_pred_bagging_nn))


Prediciendo con modelo 1/5...
Prediciendo con modelo 2/5...
Prediciendo con modelo 3/5...
Prediciendo con modelo 4/5...
Prediciendo con modelo 5/5...
F1 macro (bagging de redes): 0.013939310899704663
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.19      0.80      0.31       747
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00       133
           4       0.00      0.00      0.00        63
           5       0.15      0.19      0.17       635
           6       0.00      0.00      0.00       329
           7       0.00      0.00      0.00        46
           8       0.00      0.00      0.00        19
           9       0.00      0.00      0.00        10
          10       0.00      0.00      0.00       190
          11       0.00      0.00      0.00       261
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

### Resultados del bagging con redes neuronales

Al evaluar el ensemble de 5 redes sobre el conjunto de test obtuvimos:

- **Accuracy global:** ≈ 0,18  
- **F1 macro:** ≈ 0,014  
- **Precision macro:** ≈ 0,01  
- **Recall macro:** ≈ 0,03  

#### Comportamiento por clase

El *classification report* muestra que los errores no se reparten de forma uniforme:

- La **clase 1**, que es la combinación de enfermedades más frecuente (747 casos en el test), es prácticamente la única que el modelo identifica con cierta calidad:  
  - *Precision* ≈ 0,19  
  - *Recall* ≈ 0,80  
  - *F1-score* ≈ 0,31  

  Es decir, el ensemble suele “apostar” por esta clase y acierta en muchos de los casos que realmente pertenecen a ella, pero también etiqueta como 1 bastantes observaciones de otras clases.

- Para la mayoría de las **clases minoritarias** (0, 2, 3, 4, 6, 7, 8, …) la *precision*, el *recall* y el *F1-score* son prácticamente **0**.  
  El modelo casi nunca las predice: concentra sus decisiones en unas pocas clases grandes.

Como el **F1 macro** promedia por igual todas las clases, incluyendo aquellas en las que el rendimiento es nulo, el valor final cae a ≈ 0,014 a pesar de que la clase mayoritaria tiene un F1 razonable.

#### Comparación con la red neuronal individual

El bagging con redes neuronales **no mejora de forma relevante** el rendimiento respecto a la red neuronal individual: los valores de accuracy y F1 macro son prácticamente los mismos en ambos casos. Esto tiene sentido por varios motivos:

1. **Misma información de entrada**  
   Tanto la red individual como las redes del ensemble usan exactamente las mismas variables explicativas (sin las enfermedades originales). Si esas features no separan bien las clases, ningún método de ensamble va a “inventar” información nueva.

2. **Misma arquitectura y mismo tipo de error**  
   Todas las redes del bagging comparten la misma arquitectura (64–32 neuronas con ReLU y dropout). Como consecuencia, tienden a cometer **errores muy parecidos**: favorecen las clases más frecuentes y casi nunca detectan las raras.  
   El bagging reduce la varianza entre modelos, pero aquí el problema principal es el **sesgo**: el modelo no tiene capacidad/información suficiente para distinguir bien todas las combinaciones.

3. **Fuerte desbalance de clases**  
   El dataset está muy desbalanceado: algunas combinaciones de enfermedades aparecen muchas veces y otras muy pocas. El ensemble aprende a priorizar las clases grandes (de ahí el recall alto de la clase 1) y sacrifica completamente las minoritarias, que quedan con F1 ≈ 0.

En conjunto, este experimento muestra que el límite del rendimiento no está tanto en usar una red sola o un ensemble, sino en la propia estructura del problema y en el desbalance extremo de las clases. Con las variables disponibles, resulta difícil lograr un buen F1 macro para todas las combinaciones de enfermedades.



# Modelo de prueba



#### Descripción del modelo

Hasta ahora habíamos trabajado con un enfoque **multiclase**, donde la variable objetivo `disease_class_final` codifica cada combinación posible de enfermedades. Para este tercer modelo decidimos replantear el problema de una forma más natural: como una **clasificación multi-etiqueta**.

En este enfoque la red no intenta predecir una sola clase global, sino **seis etiquetas binarias a la vez**, una por cada enfermedad:

- `Enfermedad cardiovascular_bin`  
- `Diabetes_bin`  
- `Asma_bin`  
- `Cáncer_bin`  
- `Obesidad_bin`  
- `Depresión/Ansiedad_bin`

La arquitectura que usamos es muy parecida a la de los modelos anteriores:

- Capa de entrada con todas las variables explicativas (demográficas, estilo de vida, etc.) ya escaladas.  
- Capa densa de **64 neuronas** con activación *ReLU*.  
- Capa de **Dropout(0.3)** para evitar sobreajuste.  
- Capa densa de **32 neuronas** con activación *ReLU*.  
- Segunda capa de **Dropout(0.3)**.  
- Capa de salida con **6 neuronas** y activación **sigmoid**, una probabilidad entre 0 y 1 para cada enfermedad.

La red se entrena con:

- **Función de pérdida:** `binary_crossentropy`, adecuada para problemas multi-etiqueta donde cada salida es un 0/1 independiente.  
- **Optimizador:** `adam`.  
- **Métrica de entrenamiento:** `accuracy` (se monitorea pero las conclusiones las sacamos sobre todo con F1 por etiqueta).

Durante el entrenamiento (30 épocas) la *loss* de entrenamiento baja de forma moderada y la *loss* de validación se estabiliza alrededor de 0,47, con una accuracy de validación cercana al 18–19%. No se observa un sobreajuste extremo, pero tampoco una mejora espectacular, lo cual ya anticipa que el modelo tiene dificultades para separar bien los casos positivos de los negativos.





In [32]:
from sklearn.utils import class_weight
import numpy as np

# calculo pesos por clase en base al y_train
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

# lo convierto a diccionario {clase: peso}
class_weights = {i: w for i, w in enumerate(class_weights)}
class_weights


{0: np.float64(2.3296447291788),
 1: np.float64(0.15754544201343074),
 2: np.float64(2.6586905948820205),
 3: np.float64(0.8879023307436182),
 4: np.float64(1.8674136321195145),
 5: np.float64(0.18527095877721167),
 6: np.float64(0.35813412122840005),
 7: np.float64(2.5300442757748263),
 8: np.float64(6.2745098039215685),
 9: np.float64(11.477761836441895),
 10: np.float64(0.6183813867202598),
 11: np.float64(0.4516201874223778),
 12: np.float64(19.607843137254903),
 13: np.float64(1.5634160641000587),
 14: np.float64(3.676470588235294),
 15: np.float64(3.988035892323031),
 16: np.float64(1.3181743285549514),
 17: np.float64(0.30959752321981426),
 18: np.float64(0.807183937039653),
 19: np.float64(2.0460358056265986),
 20: np.float64(1.1619462599854757),
 21: np.float64(5.669737774627923),
 22: np.float64(0.5697194131890044),
 23: np.float64(1.3144922773578704),
 24: np.float64(3.988035892323031),
 25: np.float64(4.12796697626419),
 26: np.float64(9.603841536614645),
 27: np.float64(10

In [43]:
history = model.fit(
    X_train_scaled,
    y_train_cat,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    verbose=2,
    class_weight=class_weights
)


Epoch 1/5
100/100 - 0s - 2ms/step - accuracy: 0.0334 - loss: 2.7781 - val_accuracy: 0.0069 - val_loss: 3.4076
Epoch 2/5
100/100 - 0s - 1ms/step - accuracy: 0.0340 - loss: 2.7703 - val_accuracy: 0.0069 - val_loss: 3.4237
Epoch 3/5
100/100 - 0s - 1ms/step - accuracy: 0.0322 - loss: 2.8149 - val_accuracy: 0.0066 - val_loss: 3.4208
Epoch 4/5
100/100 - 0s - 1ms/step - accuracy: 0.0315 - loss: 2.8093 - val_accuracy: 0.0072 - val_loss: 3.4331
Epoch 5/5
100/100 - 0s - 1ms/step - accuracy: 0.0329 - loss: 2.7951 - val_accuracy: 0.0069 - val_loss: 3.4198


In [35]:
import numpy as np
import pandas as pd

n_models = 10
epocas_por_modelo = 20
n_por_clase = 200   # puedes jugar con este número

modelos_bagging = []

for i in range(n_models):
    print(f"\nEntrenando modelo balanceado {i+1}/{n_models}...")

    # construyo un bootstrap balanceado
    dfs = []
    for clase in np.unique(y_train):
        idx_clase = np.where(y_train.values == clase)[0]
        # si hay menos de n_por_clase, remuestro con reemplazo
        muestras = np.random.choice(idx_clase, size=min(len(idx_clase), n_por_clase), replace=True)
        dfs.append(pd.DataFrame({
            "y": y_train.values[muestras],
            "idx": muestras
        }))
    df_boot = pd.concat(dfs, ignore_index=True)

    idx_boot = df_boot["idx"].values
    y_boot = df_boot["y"].values

    X_boot = X_train_scaled[idx_boot]
    y_boot_cat = to_categorical(y_boot, num_classes=num_classes)

    m = crear_modelo_nn()
    m.fit(
        X_boot,
        y_boot_cat,
        epochs=epocas_por_modelo,
        batch_size=128,
        verbose=0
    )
    modelos_bagging.append(m)



Entrenando modelo balanceado 1/10...

Entrenando modelo balanceado 2/10...

Entrenando modelo balanceado 3/10...

Entrenando modelo balanceado 4/10...

Entrenando modelo balanceado 5/10...

Entrenando modelo balanceado 6/10...

Entrenando modelo balanceado 7/10...

Entrenando modelo balanceado 8/10...

Entrenando modelo balanceado 9/10...

Entrenando modelo balanceado 10/10...


# Bagging

In [ ]:
import numpy as np
import pandas as pd

n_models = 5
epocas_por_modelo = 30
n_por_clase = 100   

modelos_bagging = []

for i in range(n_models):
    print(f"\nEntrenando modelo balanceado {i+1}/{n_models}...")

    # construyo un bootstrap balanceado
    dfs = []
    for clase in np.unique(y_train):
        idx_clase = np.where(y_train.values == clase)[0]
        # si hay menos de n_por_clase, remuestro con reemplazo
        muestras = np.random.choice(idx_clase, size=min(len(idx_clase), n_por_clase), replace=True)
        dfs.append(pd.DataFrame({
            "y": y_train.values[muestras],
            "idx": muestras
        }))
    df_boot = pd.concat(dfs, ignore_index=True)

    idx_boot = df_boot["idx"].values
    y_boot = df_boot["y"].values

    X_boot = X_train_scaled[idx_boot]
    y_boot_cat = to_categorical(y_boot, num_classes=num_classes)

    m = crear_modelo_nn()
    m.fit(
        X_boot,
        y_boot_cat,
        epochs=epocas_por_modelo,
        batch_size=128,
        verbose=0
    )
    modelos_bagging.append(m)



Entrenando modelo balanceado 1/5...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Entrenando modelo balanceado 2/5...

Entrenando modelo balanceado 3/5...

Entrenando modelo balanceado 4/5...

Entrenando modelo balanceado 5/5...


In [45]:
target_cols_multi = [
    "Enfermedad cardiovascular_bin",
    "Diabetes_bin",
    "Asma_bin",
    "Cáncer_bin",
    "Obesidad_bin",
    "Depresión/Ansiedad_bin"
]

X_multi = X.copy()           # las mismas features que ya preparaste
y_multi = df[target_cols_multi].values  # matriz (n_samples, 6)

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi,
    test_size=0.2,
    random_state=42
)

scaler_m = StandardScaler()
X_train_m_scaled = scaler_m.fit_transform(X_train_m)
X_test_m_scaled  = scaler_m.transform(X_test_m)


In [46]:
num_features = X_train_m_scaled.shape[1]
num_outputs  = y_train_m.shape[1]

model_multi = Sequential()
model_multi.add(Dense(64, activation="relu", input_shape=(num_features,)))
model_multi.add(Dropout(0.3))
model_multi.add(Dense(32, activation="relu"))
model_multi.add(Dropout(0.3))
model_multi.add(Dense(num_outputs, activation="sigmoid"))  # OJO: sigmoid aquí

model_multi.compile(
    optimizer="adam",
    loss="binary_crossentropy",   # OJO: binary aquí
    metrics=["accuracy"]
)

history = model_multi.fit(
    X_train_m_scaled,
    y_train_m,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


100/100 - 1s - 13ms/step - accuracy: 0.1861 - loss: 0.5891 - val_accuracy: 0.1822 - val_loss: 0.4898
Epoch 2/30
100/100 - 0s - 2ms/step - accuracy: 0.2155 - loss: 0.5107 - val_accuracy: 0.1797 - val_loss: 0.4797
Epoch 3/30
100/100 - 0s - 2ms/step - accuracy: 0.2075 - loss: 0.4943 - val_accuracy: 0.1797 - val_loss: 0.4771
Epoch 4/30
100/100 - 0s - 2ms/step - accuracy: 0.1991 - loss: 0.4893 - val_accuracy: 0.1797 - val_loss: 0.4758
Epoch 5/30
100/100 - 0s - 2ms/step - accuracy: 0.1945 - loss: 0.4832 - val_accuracy: 0.1797 - val_loss: 0.4756
Epoch 6/30
100/100 - 0s - 1ms/step - accuracy: 0.1945 - loss: 0.4810 - val_accuracy: 0.1797 - val_loss: 0.4744
Epoch 7/30
100/100 - 0s - 1ms/step - accuracy: 0.1919 - loss: 0.4798 - val_accuracy: 0.1797 - val_loss: 0.4740
Epoch 8/30
100/100 - 0s - 1ms/step - accuracy: 0.1892 - loss: 0.4780 - val_accuracy: 0.1797 - val_loss: 0.4745
Epoch 9/30
100/100 - 0s - 1ms/step - accuracy: 0.1900 - loss: 0.4773 - val_accuracy: 0.1797 - val_loss: 0.4737
Epoch 10/30

In [47]:
y_pred_proba_m = model_multi.predict(X_test_m_scaled)
y_pred_m = (y_pred_proba_m >= 0.5).astype(int)

from sklearn.metrics import f1_score

for i, col in enumerate(target_cols_multi):
    print(col, "F1:", f1_score(y_test_m[:, i], y_pred_m[:, i]))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
Enfermedad cardiovascular_bin F1: 0.0
Diabetes_bin F1: 0.0
Asma_bin F1: 0.0
Cáncer_bin F1: 0.0
Obesidad_bin F1: 0.0
Depresión/Ansiedad_bin F1: 0.6893236074270557


In [48]:
# proporción de 1s por enfermedad en el conjunto de entrenamiento
for col in target_cols_multi:
    print(col, "positivos:", df[col].mean())


Enfermedad cardiovascular_bin positivos: 0.3362
Diabetes_bin positivos: 0.195
Asma_bin positivos: 0.1169
Cáncer_bin positivos: 0.0174
Obesidad_bin positivos: 0.2537
Depresión/Ansiedad_bin positivos: 0.5423



#### Resultados y conclusión

Para evaluar el modelo usamos el conjunto de test y calculamos el **F1-score por enfermedad** (comparando la predicción binaria con las columnas `*_bin` reales). Los resultados son:

- Enfermedad cardiovascular: **F1 ≈ 0,0**  
- Diabetes: **F1 ≈ 0,0**  
- Asma: **F1 ≈ 0,0**  
- Cáncer: **F1 ≈ 0,0**  
- Obesidad: **F1 ≈ 0,0**  
- Depresión/Ansiedad: **F1 ≈ 0,69**

Es decir, la red **solo consigue un rendimiento razonable en Depresión/Ansiedad**, mientras que prácticamente nunca acierta positivos en el resto de enfermedades (equivale a predecir casi todo como 0).

Si miramos la proporción de positivos en el conjunto de datos se entiende por qué pasa esto:

- Enfermedad cardiovascular: ~**33,6%** de positivos  
- Diabetes: ~**19,5%** de positivos  
- Asma: ~**11,7%** de positivos  
- Cáncer: solo ~**1,7%** de positivos  
- Obesidad: ~**25,4%** de positivos  
- Depresión/Ansiedad: ~**54,2%** de positivos  

Depresión/Ansiedad es claramente la condición más frecuente, mientras que Cáncer es muy rara y las demás se mueven en rangos intermedios. En este contexto, al modelo “le sale rentable” aprender bien la etiqueta más común y **ser muy conservador** con las demás: si casi siempre predice ausencia de enfermedad, la pérdida global no se dispara, pero el coste es que las etiquetas minoritarias quedan totalmente desatendidas (de ahí esos F1 ≈ 0).

En resumen, este modelo multi-etiqueta nos deja varias ideas claras:

- La red **sí es capaz de capturar cierta señal** para depresión/ansiedad (F1 ≈ 0,69), lo que sugiere que las variables disponibles contienen información útil para esa condición.  
- Para las demás enfermedades, especialmente las menos frecuentes como Cáncer, el modelo **no alcanza a aprender patrones robustos** y termina prediciendo casi siempre 0.  
- El problema de fondo vuelve a ser el mismo que veíamos en los modelos multiclase y en el bagging: el **fuerte desbalance de las etiquetas** y la falta de ejemplos suficientes para muchas enfermedades.

Por tanto, el modelo multi-etiqueta aporta una visión más detallada (podemos ver enfermedad por enfermedad), pero confirma la misma conclusión general del proyecto: con el dataset tal como está, el rendimiento de las redes neuronales está limitado sobre todo por la distribución de los datos y no tanto por la arquitectura concreta del modelo.